In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from model import ResNet20
from Train import model_training

In [ ]:
# Hyper parameters
epoch_losses = []
train_loss_list = []
epochs = 50
lr = 0.001
batch_size = 64

# Model device assignment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet20().to(device)

In [ ]:
# Define transformation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465),
                            (0.2023, 0.1994, 0.2010))])

# Load the training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

# Load the test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# Define the class names for reference
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Testing model on Cifar10
testing_module = model_training(model, lr, trainloader, epochs)
train_loss = testing_module.forward(device)

Start Training for 50 epochs
Epoch [1/50], Average Loss: 1.8471
Epoch [2/50], Average Loss: 1.7601
Epoch [3/50], Average Loss: 1.7113
Epoch [4/50], Average Loss: 1.6712
Epoch [5/50], Average Loss: 1.6480
Epoch [6/50], Average Loss: 1.6179


In [ ]:
# Visualization of training results 
plt.plot(train_loss, linewidth=2, c='r')
plt.xlabel("Epochs")
plt.ylabel("Training Loss")
plt.title("Training Loss of ResNet20 Model")
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
with torch.no_grad():
    number_correct = 0
    samples = 0
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, prediction = torch.max(outputs, 1)
        samples += labels.size(0)
        number_correct += (prediction == labels).sum().item()

accuracy = (number_correct / samples) * 100.0